In [1]:
from typing import OrderedDict
import syft as sy
from syft.core.node.common.node_service.tff.tff_messages import TFFMessageWithReply

import os, sys

import collections

import numpy as np
import tensorflow as tf
import tensorflow_federated as tff    
import io
from tensorflow_federated.python.learning.model_utils import ModelWeights 


# np.random.seed(0)

/home/teo/anaconda3/envs/PySyTFF/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
domain = sy.login(email="info@openmined.org", password="changethis", port=8081)


Anyone can login as an admin to your node right now because your password is still the default PySyft username and password!!!

Connecting to localhost... done! 	 Logging into practical_krizhevsky... done!


In [3]:
domain.datasets


Idx,Name,Description,Assets,Id
[0],Mixed MedNIST,Small dataset for TFF testing,"[""images""] -> [""labels""] ->",8fd185de-cdde-4785-a457-cdf65314b364


In [4]:
def create_keras_model():
  return tf.keras.models.Sequential([
      tf.keras.layers.InputLayer(input_shape=(64*64,)),
      tf.keras.layers.Dense(6, kernel_initializer='zeros'),
      tf.keras.layers.Softmax(),
  ])


In [4]:
def remote_train_model(model_fn, dataset_id, rounds, no_clients, noise_multiplier, clients_per_round):
    tf.config.optimizer.set_experimental_options({'disable_meta_optimizer': True})

    model = model_fn()

    input_spec = (tf.TensorSpec(shape=(1,64*64), dtype=tf.int32, name=None), 
                tf.TensorSpec(shape=(1,1), dtype=tf.int32, name=None))

    functional_model = tff.learning.models.functional_model_from_keras(keras_model=model, 
                                                                    loss_fn=tf.keras.losses.SparseCategoricalCrossentropy(),
                                                                    input_spec=input_spec)
    print(functional_model.input_spec[0])
    tff.learning.models.save_functional_model(functional_model=functional_model, path='tmp')

    import shutil
    shutil.make_archive('archive', 'zip', 'tmp')
    
    params = {
    'rounds': rounds,
    'dataset_id': dataset_id,
    'no_clients': no_clients,
    'noise_multiplier': noise_multiplier,
    'clients_per_round': clients_per_round,
    }

    with open('archive.zip', 'rb') as f:
        model_bytes = f.read()
    # model_bytes = None

    # params = {}
    msg = TFFMessageWithReply(params=params, model_bytes=model_bytes)
    reply_msg = domain.send_immediate_msg_with_reply(msg)
    
    payload = eval(reply_msg.payload)
    
    memfile = io.BytesIO()
    memfile.write(payload['trainable'])
    memfile.seek(0)
    trainable = np.load(memfile, allow_pickle=True)
    
    memfile2 = io.BytesIO()
    memfile2.write(payload['non_trainable'])
    memfile2.seek(0)
    non_trainable = np.load(memfile2, allow_pickle=True)

    modelweights = ModelWeights(list(trainable), list(non_trainable))
    model = model_fn()
    modelweights.assign_weights_to(model)
    return model

In [5]:
model_fn = create_keras_model 
params = {
    'rounds': 10,
    'dataset_id': domain.datasets[0].id,
    'no_clients': 5,
    'noise_multiplier': 0.05,
    'clients_per_round': 2,
    }
 
model = sy.tff.train_model(model_fn, params, domain)

TensorSpec(shape=(1, 4096), dtype=tf.int64, name=None)


INFO:tensorflow:Assets written to: _tmp/assets


INFO:tensorflow:Assets written to: _tmp/assets
[2022-07-26T19:18:20.463972+0300][CRITICAL][logger]][183803] UnknownPrivateException has been triggered.


UnknownPrivateException: UnknownPrivateException has been triggered.

In [10]:
model

In [ ]:
# download MedNIST.pkl
if not os.path.exists("./MedNIST.pkl"):
    os.system('curl -O "https://media.githubusercontent.com/media/shubham3121/datasets/main/MedNIST/MedNIST.pkl"')
else:
    print("MedNIST already downloaded")

In [ ]:
# Helper Methods

import os
import json
import pandas as pd
from PIL import Image
from enum import Enum
from collections import defaultdict
import numpy as np
from syft.core.adp.data_subject_list import DataSubjectList


def get_label_mapping():
    # the data uses the following mapping
    mapping = {
        "AbdomenCT": 0, 
        "BreastMRI": 1, 
        "CXR": 2, 
        "ChestCT": 3, 
        "Hand": 4, 
        "HeadCT": 5
    }
    return mapping

df = pd.read_pickle("./MedNIST.pkl")
mapping = get_label_mapping()

total_num = df.shape[0]
print("Columns:", df.columns)
print("Total Images:", total_num)
print("Label Mapping", mapping)

Columns: Index(['patient_id', 'image', 'label'], dtype='object')
Total Images: 58954
Label Mapping {'AbdomenCT': 0, 'BreastMRI': 1, 'CXR': 2, 'ChestCT': 3, 'Hand': 4, 'HeadCT': 5}


In [ ]:
x = df['image'][10000]
x = tf.reshape(x, [-1,64*64])

In [ ]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  try:
    tf.config.experimental.set_virtual_device_configuration(
        gpus[0],[tf.config.experimental.VirtualDeviceConfiguration(memory_limit=5120)])
  except RuntimeError as e:
    print(e)

Virtual devices cannot be modified after being initialized


In [ ]:
model.predict(x)

1/1 [==============================] - 1s 690ms/step


array([[1., 0., 0., 0., 0., 0.]], dtype=float32)

In [ ]:
tf.test.is_gpu_available(
    cuda_only=False, min_cuda_compute_capability=None
)

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


True